In [25]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
# Load the model from the file
model = load_model('clark-y-cnn.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               512       
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                                 
Total params: 11,539
Trainable params: 11,539
Non-traina

In [27]:
# Load the data
df = pd.read_csv('clark-y-2.csv')

# Filter the data
filtered_df = df[
    (df['Reynold'] >= 0.1e5) & (df['Reynold'] <= 40e5) &  # 雷诺数范围
    (df['Mach'] >= 0.1) & (df['Mach'] <= 0.7) &            # 马赫数范围
    (df['Alpha'] >= -3) & (df['Alpha'] <= 15)              # 迎角范围
]

# Split the data into inputs (X) and outputs (y)
X = filtered_df[['Reynold', 'Mach', 'Alpha']].values
y = filtered_df[['cl', 'cd', 'cm']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 29)

In [28]:
# Define scaler 
scaler = StandardScaler()
scalery = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scalery.fit_transform(y_train)
y_test = scalery.transform(y_test)

In [31]:
# Desired alpha
desire_re = 2636315
desire_machs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7000000000000001]
alpha_values = np.linspace(-3, 15, 100)

# new X
for i in range(len(desire_machs)):
    desire_mach = desire_machs[i]
    new_X = np.array([[desire_re, desire_mach, alpha] for alpha in alpha_values])
    new_X_scaled = scaler.transform(new_X)

    # use model to predict new values
    predictions = model.predict(new_X_scaled)
    predictions = scalery.inverse_transform(predictions)

    # extract predicted Cl, Cd, Cm
    predicted_cl = predictions[:, 0]
    predicted_cd = predictions[:, 1]
    predicted_cm = predictions[:, 2]

    # original data
    filtered_df = df[(df['Reynold'] == desire_re) & (df['Mach'] == desire_mach)]
    original_alpha = filtered_df['Alpha'].values
    original_cl = filtered_df['cl'].values
    original_cd = filtered_df['cd'].values
    original_cm = filtered_df['cm'].values

    # plotting
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)    # Cl
    plt.plot(alpha_values, predicted_cl, label='Predicted Cl', color='blue')
    plt.scatter(original_alpha, original_cl, label='Original Cl', color='orange')
    plt.title(f'Re = {desire_re:.0f} Mach = {desire_mach:.2f} \n Cl vs Alpha')
    plt.xlabel('Alpha')
    plt.ylabel('Cl')
    plt.legend()

    plt.subplot(1, 3, 2)    # Cd
    plt.plot(alpha_values, predicted_cd, label='Predicted Cd', color='blue')
    plt.scatter(original_alpha, original_cd, label='Original Cd', color='orange')
    plt.title(f'Re = {desire_re:.0f} Mach = {desire_mach:.2f} \n Cd vs Alpha')
    plt.xlabel('Alpha')
    plt.ylabel('Cd')
    plt.legend()

    plt.subplot(1, 3, 3)    # Cm
    plt.plot(alpha_values, predicted_cm, label='Predicted Cm', color='blue')
    plt.scatter(original_alpha, original_cm, label='Original Cm', color='orange')
    plt.title(f'Re = {desire_re:.0f} Mach = {desire_mach:.2f} \n Cm vs Alpha')
    plt.xlabel('Alpha')
    plt.ylabel('Cm')
    plt.legend()
    
    plt.tight_layout()

    # Save as SVG
    plt.savefig(f'plot_Re{desire_re:.0f}_Mach{desire_mach:.2f}.svg', format='svg')
    plt.close()

4/4 [==============================] - 0s 19ms/step
